In [463]:
import pandas as pd
import json
res_path='../out_results/clip'

In [464]:
def get_results_df(jsonl_f, skip_svm=False, skip_Pichunter=False):
    data = []

    # Read JSONL file line by line and parse each line as JSON
    with open(jsonl_f, 'r') as f:
        for line in f:
            line_data = json.loads(line.strip())
            if skip_svm and 'SVM' in line_data['method'] and  line_data['use_batch'] == False:
                continue
            if skip_Pichunter and 'Pichunter' in line_data['method']:
                continue
            data.append(line_data)
    # Convert list of dictionaries to DataFrame
    df_tmp = pd.DataFrame(data)

    print(f"{jsonl_f} shape, {df_tmp.shape}")
    return df_tmp

In [465]:
#jsonl_file= f'{res_path}/exps_results_grid_search_svm_incr.jsonl'
#jsonl_files= [f'{res_path}/exps_results_grid_search_rocchio.jsonl',f'{res_path}/exps_results_grid_search_rocchio_batch.jsonl']à#
#jsonl_files= [f'{res_path}/exps_results_grid_search_poly.jsonl',f'{res_path}/exps_results_grid_search_poly_batch.jsonl']
jsonl_files= [f'{res_path}/exps_results_grid_search_poly_msed.jsonl',f'{res_path}/exps_results_grid_search_poly_msed_batch.jsonl']
dfs=[]
for jsonl_file in jsonl_files:
    dfs.append(get_results_df(jsonl_file))
df=pd.concat(dfs)
print(df['dataset'].unique())
print(df['seed'].unique())

../out_results/clip/exps_results_grid_search_poly_msed.jsonl shape, (41600, 26)
../out_results/clip/exps_results_grid_search_poly_msed_batch.jsonl shape, (41600, 26)
['df_1608' 'df_1707' 'df_1597' 'df_1671' 'df_1598' 'df_1714' 'df_1711'
 'df_1746' 'df_1725' 'df_1678' 'df_1739' 'df_1606' 'df_1733' 'df_1710'
 'df_1676' 'df_1720' 'df_1736' 'df_1747' 'df_1744' 'df_1713' 'df_1593'
 'df_1738' 'df_1750' 'df_1728' 'df_1672' 'df_1719' 'df_1702' 'df_1717'
 'df_1748' 'df_1664' 'df_1665' 'df_1667' 'df_1669' 'df_1741' 'df_1609'
 'df_1701' 'df_1729' 'df_1724' 'df_1607' 'df_1715' 'df_1675' 'df_1591'
 'df_1668' 'df_1735' 'df_1670' 'df_1677' 'df_1673' 'df_1594' 'df_1595'
 'df_1663' 'df_1604' 'df_1705' 'df_1680' 'df_1666' 'df_1610' 'df_1712'
 'df_1732' 'df_1601' 'df_1731' 'df_1592' 'df_1703' 'df_1704' 'df_1706'
 'df_1734' 'df_1721' 'df_1708' 'df_1661' 'df_1726' 'df_1602' 'df_1718'
 'df_1709' 'df_1603' 'df_1740' 'df_1599' 'df_1723' 'df_1749' 'df_1737'
 'df_1745' 'df_1600' 'df_1605']
[0 1 2 3 4]


In [466]:
# Columns to drop
#df = df[df['seed'] != 4] #removing sed 4 since there are incomplete resultsfor this seed 
columns_to_drop = ['action_step', 'n_actions', 'n_pos_action', 'n_neg_action', 'actions','indexes_positive_initial_images', 'dataset', 'seed','n_pos_at_iter']
df['initial_query'] = df['initial_query'].fillna('none')
# Drop the specified columns
df = df.drop(columns=columns_to_drop)
df.head()


,iteration,recall,ndcg,map,max_iter,n_display,k_pos,k_neg,method,fun_name,data_preprocessing,initial_query,use_batch,exp_id,alpha,beta,gamma
0,0,0.10,0.228603,0.321587,3,50,3,3,Polyquery_msed_logscale,msed,softmax,none,False,0984fa097a74da56e76cf844ed8b9b243371bb9d,0.25,1.00,0.25
1,1,0.82,0.842945,0.856834,3,50,3,3,Polyquery_msed_logscale,msed,softmax,none,False,0984fa097a74da56e76cf844ed8b9b243371bb9d,0.25,1.00,0.25
2,2,1.00,1.000000,1.000000,3,50,3,3,Polyquery_msed_logscale,msed,softmax,none,False,0984fa097a74da56e76cf844ed8b9b243371bb9d,0.25,1.00,0.25
3,3,1.00,1.000000,1.000000,3,50,3,3,Polyquery_msed_logscale,msed,softmax,none,False,0984fa097a74da56e76cf844ed8b9b243371bb9d,0.25,1.00,0.25
4,0,0.10,0.228603,0.321587,3,50,3,3,Polyquery_msed_logscale,msed,softmax,none,False,dc36c530fb02b6820a26322417254c6591c9f77b,0.50,0.75,0.25


In [467]:
original_df = df.copy()


In [468]:
#'alpha','beta', 'gamma'
#group df by method	fun_name	data_preprocessing	use_batch	initial_query	max_iter	n_display	k_pos	k_neg	iteration and average the values of the  columns recall	ndcg	map
df_grouped = df.groupby(['method','fun_name', 'data_preprocessing', 'use_batch', 'alpha','beta', 'gamma','initial_query', 'max_iter', 'n_display', 'k_pos', 'k_neg', 'iteration']).agg({
    'recall': 'mean',
    'ndcg': 'mean',
    'map': 'mean'
}).reset_index()

#do teh same but compute the standard deviation
df_grouped_std = df.groupby(['method','fun_name', 'data_preprocessing', 'use_batch',  'alpha','beta', 'gamma','initial_query', 'max_iter', 'n_display', 'k_pos', 'k_neg', 'iteration']).agg({
    'recall': 'std',
    'ndcg': 'std',
    'map': 'std'
}).reset_index()

#combine the two dataframes
df_grouped = pd.merge(df_grouped, df_grouped_std, on=['method','fun_name', 'data_preprocessing', 'use_batch',  'alpha','beta', 'gamma','initial_query', 'max_iter', 'n_display', 'k_pos', 'k_neg', 'iteration'], suffixes=('_mean', '_std'))

df_grouped

,method,fun_name,data_preprocessing,use_batch,alpha,beta,gamma,initial_query,max_iter,n_display,k_pos,k_neg,iteration,recall_mean,ndcg_mean,map_mean,recall_std,ndcg_std,map_std
0,Polyquery_msed_logscale,msed,softmax,False,0.25,0.75,0.0,none,3,50,-1,0,0,0.1000,0.228603,0.321587,0.000000,0.000000,0.000000
1,Polyquery_msed_logscale,msed,softmax,False,0.25,0.75,0.0,none,3,50,-1,0,1,0.5762,0.625652,0.675588,0.251487,0.234151,0.232758
2,Polyquery_msed_logscale,msed,softmax,False,0.25,0.75,0.0,none,3,50,-1,0,2,0.5918,0.645989,0.711994,0.252900,0.231283,0.218774
3,Polyquery_msed_logscale,msed,softmax,False,0.25,0.75,0.0,none,3,50,-1,0,3,0.5918,0.645989,0.711994,0.252900,0.231283,0.218774
4,Polyquery_msed_logscale,msed,softmax,False,0.25,0.75,0.0,none,3,50,3,0,0,0.1000,0.228603,0.321587,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
203,Polyquery_msed_logscale,msed,softmax,True,1.00,1.00,1.0,none,3,50,-1,-1,3,0.6494,0.709812,0.786455,0.241913,0.210927,0.188302
204,Polyquery_msed_logscale,msed,softmax,True,1.00,1.00,1.0,none,3,50,3,3,0,0.1000,0.228603,0.321587,0.000000,0.000000,0.000000
205,Polyquery_msed_logscale,msed,softmax,True,1.00,1.00,1.0,none,3,50,3,3,1,0.5415,0.604393,0.665066,0.246962,0.221130,0.216834
206,Polyquery_msed_logscale,msed,softmax,True,1.00,1.00,1.0,none,3,50,3,3,2,0.6118,0.678917,0.757047,0.240798,0.208798,0.190941


In [469]:
save_path = jsonl_files[0].split('.jsonl')[0] + '_grouped.csv'
# Specify the data types for each column
dtype = {
    'method': str,
    'fun_name': str,
    'data_preprocessing': str,
    'use_batch': bool,
    'alpha':float,
    'beta': float, 
    'gamma': float,
    'initial_query': str,
    'max_iter': int,
    'n_display': int,
    'k_pos': int,
    'k_neg': int,
    'iteration': int,
    'recall_mean': float,
    'ndcg_mean': float,
    'map_mean': float,
    'recall_std': float,
    'ndcg_std': float,
    'map_std': float
    
}
#df_grouped['recall_mean'] = df_grouped['recall_mean'].round(3)
df_grouped[df_grouped['iteration'] == 3].to_csv(save_path, index=False)


df_grouped.shape

(208, 19)

In [470]:
# # in df grouped select only the lines that have itaration==3 and drop the max_iter column
# df_it3= df_grouped[df_grouped['iteration'] == 3].drop(columns=['max_iter', 'iteration', 'initial_query', 'n_display', ])
df_it=df_grouped[df_grouped['iteration'] == 3]
# in df_it3 we have teh following columns 
#method	alpha	beta	gamma	fun_name	data_preprocessing	use_batch	initial_query	n_display	k_pos	k_neg	recall	ndcg	map
# group df_it3 by ['method','fun_name', 'data_preprocessing', 'use_batch', 'initial_query', 'n_display', 'k_pos', 'k_neg'] amd visualize the index of teh row with maximum recall

# df_max_recall = df_it.groupby(['method','fun_name', 'data_preprocessing', 'iteration', 'use_batch', 'k_pos', 'k_neg']).agg({
#     'recall_mean': 'max'
# })
# # #select in df_it3 the lines that have the index of the rows with maximum recall
# df_final = df_max_recall.reset_index()
# df_final = df_final.merge(df_it, on=['method','fun_name', 'data_preprocessing', 'use_batch','k_pos', 'k_neg', 'iteration','recall_mean'])
# print(df_final)
# Keep only the rows with the maximum recall
#round ['recall_mean'] at 4th decimal place


df_it['max_recall'] = df_it.groupby(['method', 'fun_name', 'data_preprocessing', 'iteration', 'use_batch', 'k_pos', 'k_neg'])['recall_mean'].transform('max')
df_final = df_it[df_it['recall_mean'] == df_it['max_recall']]

save_path = jsonl_files[0].split('.jsonl')[0] + '_bestRecall.csv'
# Specify the data types for each column
dtype = {
    'method': str,
    'fun_name': str,
    'data_preprocessing': str,
    'use_batch': bool,
    'initial_query': str,
    'max_iter': int,
    'n_display': int,
    'k_pos': int,
    'k_neg': int,
    'iteration': int,
    'recall': float,
    'ndcg': float,
    'map': float
}
#print(df_final)
df_final.to_csv(save_path, index=False)
df_final.shape


#sort according to k_pos
df_final.sort_values(by='k_pos', inplace=True)
df_final[df_final['k_neg'] == -1]

/tmp/ipykernel_3580500/2240975089.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_it['max_recall'] = df_it.groupby(['method', 'fun_name', 'data_preprocessing', 'iteration', 'use_batch', 'k_pos', 'k_neg'])['recall_mean'].transform('max')
/tmp/ipykernel_3580500/2240975089.py:45: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final.sort_values(by='k_pos', inplace=True)


,method,fun_name,data_preprocessing,use_batch,alpha,beta,gamma,initial_query,max_iter,n_display,k_pos,k_neg,iteration,recall_mean,ndcg_mean,map_mean,recall_std,ndcg_std,map_std,max_recall
187,Polyquery_msed_logscale,msed,softmax,True,1.0,0.50,0.50,none,3,50,-1,-1,3,0.6494,0.709812,0.786455,0.241913,0.210927,0.188302,0.6494
179,Polyquery_msed_logscale,msed,softmax,True,1.0,0.25,0.25,none,3,50,-1,-1,3,0.6494,0.709812,0.786455,0.241913,0.210927,0.188302,0.6494
195,Polyquery_msed_logscale,msed,softmax,True,1.0,0.75,0.75,none,3,50,-1,-1,3,0.6494,0.709812,0.786455,0.241913,0.210927,0.188302,0.6494
203,Polyquery_msed_logscale,msed,softmax,True,1.0,1.00,1.00,none,3,50,-1,-1,3,0.6494,0.709812,0.786455,0.241913,0.210927,0.188302,0.6494
91,Polyquery_msed_logscale,msed,softmax,False,1.0,0.75,0.75,none,3,50,-1,-1,3,0.6258,0.680273,0.744113,0.242329,0.217386,0.202105,0.6258
83,Polyquery_msed_logscale,msed,softmax,False,1.0,0.50,0.50,none,3,50,-1,-1,3,0.6258,0.680273,0.744113,0.242329,0.217386,0.202105,0.6258
75,Polyquery_msed_logscale,msed,softmax,False,1.0,0.25,0.25,none,3,50,-1,-1,3,0.6258,0.680273,0.744113,0.242329,0.217386,0.202105,0.6258
99,Polyquery_msed_logscale,msed,softmax,False,1.0,1.00,1.00,none,3,50,-1,-1,3,0.6258,0.680273,0.744113,0.242329,0.217386,0.202105,0.6258
